In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers
# from functools import partial
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# set seed for reproducibility
random_seed = 1

In [2]:
df = pd.read_csv('fred_230718.csv', index_col='Date', parse_dates=True)
df = df.asfreq('QS')
earliest_date = '1963-01-01'
latest_date = '2021-10-01'
# # filter df index to be between earliest_date and latest_date
df = df.loc[(df.index >= earliest_date) & (df.index <= latest_date)]
df.dropna(axis=1, inplace=True)
df.head()

,GDP,GDPC1,GDPPOT,CPIAUCSL,CPILFESL,GDPDEF,M1V,M2V,DFF,UNRATE,...,MANEMP,DSPIC96,PCE,PCEDG,PSAVERT,DSPI,INDPRO,HOUST,GPDI,MSPUS
Date,,,,,,,,,,,,,,,,,,,,,
1963-01-01,621.672,3628.306,3662.738125,30.44,31.5,17.134,4.178,1.690,3.00,5.7,...,15545.0,2541.1,374.4,53.1,10.9,430.0,26.0448,1244.0,99.689,17800.0
1963-04-01,629.752,3669.020,3701.698767,30.48,31.7,17.164,4.194,1.675,3.00,5.7,...,15602.0,2547.1,376.4,53.2,10.7,431.1,26.7473,1689.0,101.650,18000.0
1963-07-01,644.444,3749.681,3741.388301,30.69,31.8,17.187,4.248,1.680,3.00,5.6,...,15646.0,2572.6,384.4,55.5,10.1,438.0,27.0445,1614.0,104.612,17900.0
1963-10-01,653.938,3774.264,3781.880559,30.75,32.0,17.326,4.269,1.672,3.50,5.5,...,15714.0,2617.3,386.0,54.2,11.5,447.0,27.5578,1779.0,107.189,18500.0
1964-01-01,669.822,3853.835,3822.450115,30.94,32.2,17.381,4.345,1.685,3.25,5.6,...,15715.0,2652.8,396.8,57.9,10.7,455.3,27.8820,1603.0,110.474,18500.0


set target and create, train, validate, and test datasets and then scale and transform them so they will work better with the neural network

In [10]:
target = 'MSPUS'
y = df[target]
X = df.drop(columns=[target]).shift(1).dropna()
y = y.loc[X.index] # Make sure y and X have the same rows after dropna

# https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=random_seed)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)  # validation data should also be scaled
X_test_scaled = scaler.transform(X_test)

# Log-transform the target variable
y_train_log = np.log1p(y_train)
y_valid_log = np.log1p(y_valid)

In [36]:
# from sklearn.preprocessing import StandardScaler

# # assuming X_train and X_test are your training and test datasets
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_valid = scaler.transform(X_valid)  # validation data should also be scaled
# X_test_scaled = scaler.transform(X_test)

# # Check the type and shape of X_train
# print("Type of X_train:", type(X_train))
# print("Shape of X_train:", X_train.shape)

# # Check the type and shape of y_train
# print("Type of y_train:", type(y_train))
# print("Shape of y_train:", y_train.shape)

# # Check the range of y_train values
# print("Min of y_train:", y_train.min())
# print("Max of y_train:", y_train.max())

# # If you're using pandas, you can get a statistical summary of your data
# if isinstance(X_train, pd.DataFrame):
#     print(X_train.describe())
    
# print(X_train)
# print(X_train)

# # Log-transform the target variable
# y_train = np.log1p(y_train)
# y_valid = np.log1p(y_valid)

Type of X_train: <class 'numpy.ndarray'>
Shape of X_train: (141, 25)
Type of y_train: <class 'pandas.core.series.Series'>
Shape of y_train: (141,)
Min of y_train: 17900.0
Max of y_train: 369800.0
                  GDP      GDPC1        GDPPOT  CPIAUCSL  CPILFESL   GDPDEF  \
Date                                                                          
1970-04-01   1051.200   4939.759   4942.234479    37.900    39.600   21.280   
2010-01-01  14651.248  15379.155  16122.163920   216.509   220.501   95.267   
1975-04-01   1616.116   5551.713   5821.185285    52.300    52.300   29.110   
1984-10-01   4084.250   7690.985   7828.967378   104.100   105.000   53.104   
2002-07-01  10887.460  13477.356  13739.879490   179.300   189.700   80.783   
...               ...        ...           ...       ...       ...      ...   
1983-04-01   3473.413   6896.561   7442.965382    97.900    97.600   50.364   
1974-01-01   1476.289   5731.632   5579.493499    45.600    46.300   25.757   
2001-01-01  10

let's create functions to train and evaluate models

In [11]:
def train_model(X_train, y_train, 
                X_valid, y_valid,
                layer_sizes=[100, 100], 
                activation="relu", 
                kernel_initializer="he_normal", 
                learning_rate=0.001, 
                epochs=100,
                batch_norm=False,
                l1_l2=False,
                l1=.01,
                l2=.01):

    # Create a sequential model
    model = tf.keras.Sequential()

    # Add batch normalization and dense layers according to the layer_sizes
    for size in layer_sizes:
        if batch_norm:
            model.add(tf.keras.layers.BatchNormalization())
        if l1_l2:
            model.add(tf.keras.layers.Dense(size, activation=activation, kernel_initializer=kernel_initializer
                                            , kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))
        else:
            model.add(tf.keras.layers.Dense(size, activation=activation, kernel_initializer=kernel_initializer))

    # Add a final Dense layer with no activation
    model.add(tf.keras.layers.Dense(1))

    # Create the optimizer with the custom learning rate
    sgd = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    # Compile the model
    model.compile(loss="mse", optimizer=sgd)

    # Train the model using the scaled data
    model.fit(X_train, y_train, epochs=epochs, validation_data=(X_valid, y_valid))
    
    return model

In [17]:
def evaluate_model(model, X_test, y_test, log_target=False):
    # When predicting, transform the predictions back
    y_pred = model.predict(X_test)
    if log_target:
        y_pred = np.expm1(y_pred).flatten()  # inverse of np.log1p(), make it 1D


    # compute the RMSE on the original scale
    mse = np.mean(tf.keras.losses.MSE(y_test, y_pred))
    print('Test set MSE:', mse)
    rmse = np.sqrt(mse)
    print('Test set RMSE:', rmse)
    rmspe = (np.sqrt(np.mean(np.square((y_test - y_pred) / y_test)))) * 100
    print('Test set RMSPE (%):', rmspe)

In [77]:
# def train_model(X_train, y_train, 
#                 X_valid, y_valid,
#                 layer_sizes=[100, 100], 
#                 activation="relu", 
#                 kernel_initializer="he_normal", 
#                 learning_rate=0.001, 
#                 epochs=100,
#                 batch_norm=False,
#                 l1_l2=False,
#                 l1=.01,
#                 l2=.01):

#     # Create a sequential model
#     model = tf.keras.Sequential()

#     # Add batch normalization and dense layers according to the layer_sizes
#     for size in layer_sizes:
#         if batch_norm:
#             model.add(tf.keras.layers.BatchNormalization())
#         if l1_l2:
#             model.add(tf.keras.layers.Dense(size, activation=activation, kernel_initializer=kernel_initializer
#                                             , kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))
#         else:
#             model.add(tf.keras.layers.Dense(size, activation=activation, kernel_initializer=kernel_initializer))

#     # Add a final Dense layer with no activation
#     model.add(tf.keras.layers.Dense(1))

#     # Create the optimizer with the custom learning rate
#     sgd = tf.keras.optimizers.SGD(learning_rate=learning_rate)

#     # Compile the model
#     model.compile(loss="mse", optimizer=sgd)

#     # Train the model using the scaled data
#     model.fit(X_train, y_train, epochs=epochs, validation_data=(X_valid, y_valid))
    
#     return model

Now we'll test some different architectures

In [19]:
print('no regularization')
model = train_model(X_train=X_train_scaled, y_train=y_train_log, X_valid=X_test_scaled, y_valid=y_valid_log)
evaluate_model(model, X_test_scaled, y_test, log_target=True)

no regularization
Epoch 1/100
5/5 [==============================] - 0s 30ms/step - loss: 70.5498 - val_loss: 18.2292
Epoch 2/100
5/5 [==============================] - 0s 9ms/step - loss: 13.0595 - val_loss: 9.4761
Epoch 3/100
5/5 [==============================] - 0s 9ms/step - loss: 7.7773 - val_loss: 7.1266
Epoch 4/100
5/5 [==============================] - 0s 8ms/step - loss: 5.4752 - val_loss: 6.2495
Epoch 5/100
5/5 [==============================] - 0s 9ms/step - loss: 4.0522 - val_loss: 5.7089
Epoch 6/100
5/5 [==============================] - 0s 8ms/step - loss: 3.2820 - val_loss: 5.4086
Epoch 7/100
5/5 [==============================] - 0s 9ms/step - loss: 2.7394 - val_loss: 5.0812
Epoch 8/100
5/5 [==============================] - 0s 9ms/step - loss: 2.4031 - val_loss: 4.7386
Epoch 9/100
5/5 [==============================] - 0s 9ms/step - loss: 2.1108 - val_loss: 4.6396
Epoch 10/100
5/5 [==============================] - 0s 9ms/step - loss: 1.8883 - val_loss: 4.4839
Epoch 1

In [20]:
print('batch norm regularization')
model = train_model(X_train=X_train_scaled, y_train=y_train_log, X_valid=X_test_scaled, y_valid=y_valid_log, batch_norm=True)
evaluate_model(model, X_test_scaled, y_test, log_target=True)

batch norm regularization
Epoch 1/100
5/5 [==============================] - 1s 34ms/step - loss: 84.8679 - val_loss: 49.5498
Epoch 2/100
5/5 [==============================] - 0s 10ms/step - loss: 33.2674 - val_loss: 28.4453
Epoch 3/100
5/5 [==============================] - 0s 10ms/step - loss: 13.9458 - val_loss: 20.3044
Epoch 4/100
5/5 [==============================] - 0s 10ms/step - loss: 6.3257 - val_loss: 16.5661
Epoch 5/100
5/5 [==============================] - 0s 10ms/step - loss: 3.0580 - val_loss: 15.1796
Epoch 6/100
5/5 [==============================] - 0s 10ms/step - loss: 3.0581 - val_loss: 13.7792
Epoch 7/100
5/5 [==============================] - 0s 11ms/step - loss: 2.1410 - val_loss: 14.1509
Epoch 8/100
5/5 [==============================] - 0s 10ms/step - loss: 3.8369 - val_loss: 13.6924
Epoch 9/100
5/5 [==============================] - 0s 11ms/step - loss: 2.1171 - val_loss: 13.6655
Epoch 10/100
5/5 [==============================] - 0s 10ms/step - loss: 1.8299 

In [21]:
print('L1/L2 regularization')
model = train_model(X_train=X_train_scaled, y_train=y_train_log, X_valid=X_test_scaled, y_valid=y_valid_log, l1_l2=True)
evaluate_model(model, X_test_scaled, y_test, log_target=True)

L1/L2 regularization
Epoch 1/100
5/5 [==============================] - 0s 33ms/step - loss: 120.7640 - val_loss: 42.7649
Epoch 2/100
5/5 [==============================] - 0s 9ms/step - loss: 35.6855 - val_loss: 31.3888
Epoch 3/100
5/5 [==============================] - 0s 10ms/step - loss: 29.0362 - val_loss: 29.2702
Epoch 4/100
5/5 [==============================] - 0s 9ms/step - loss: 26.8305 - val_loss: 27.5348
Epoch 5/100
5/5 [==============================] - 0s 9ms/step - loss: 25.5189 - val_loss: 26.9305
Epoch 6/100
5/5 [==============================] - 0s 9ms/step - loss: 24.7070 - val_loss: 26.4374
Epoch 7/100
5/5 [==============================] - 0s 9ms/step - loss: 24.1821 - val_loss: 26.0597
Epoch 8/100
5/5 [==============================] - 0s 9ms/step - loss: 23.8158 - val_loss: 25.5782
Epoch 9/100
5/5 [==============================] - 0s 8ms/step - loss: 23.4779 - val_loss: 25.3300
Epoch 10/100
5/5 [==============================] - 0s 9ms/step - loss: 23.2357 - val

In [22]:
print('batch norm & L1/L2 regularization')
model = train_model(X_train=X_train_scaled, y_train=y_train_log, X_valid=X_test_scaled, y_valid=y_valid_log, batch_norm=True, l1_l2=True)
evaluate_model(model, X_test_scaled, y_test, log_target=True)

batch norm & L1/L2 regularization
Epoch 1/100
5/5 [==============================] - 1s 40ms/step - loss: 132.4064 - val_loss: 99.2007
Epoch 2/100
5/5 [==============================] - 0s 11ms/step - loss: 67.2103 - val_loss: 70.9417
Epoch 3/100
5/5 [==============================] - 0s 11ms/step - loss: 40.0420 - val_loss: 57.1364
Epoch 4/100
5/5 [==============================] - 0s 12ms/step - loss: 30.2002 - val_loss: 49.4219
Epoch 5/100
5/5 [==============================] - 0s 11ms/step - loss: 25.4513 - val_loss: 44.9561
Epoch 6/100
5/5 [==============================] - 0s 10ms/step - loss: 25.1547 - val_loss: 43.2461
Epoch 7/100
5/5 [==============================] - 0s 11ms/step - loss: 24.4969 - val_loss: 41.6827
Epoch 8/100
5/5 [==============================] - 0s 10ms/step - loss: 24.8143 - val_loss: 41.1456
Epoch 9/100
5/5 [==============================] - 0s 11ms/step - loss: 23.8803 - val_loss: 40.4307
Epoch 10/100
5/5 [==============================] - 0s 11ms/step 

In [23]:
model = train_model(epochs=500, layer_sizes=[100, 100, 100], X_train=X_train_scaled, y_train=y_train_log, X_valid=X_test_scaled, y_valid=y_valid_log, batch_norm=True, l1_l2=True)
evaluate_model(model, X_test_scaled, y_test, log_target=True)

Epoch 1/500
5/5 [==============================] - 1s 48ms/step - loss: 130.8462 - val_loss: 117.1993
Epoch 2/500
5/5 [==============================] - 0s 11ms/step - loss: 71.6521 - val_loss: 86.0730
Epoch 3/500
5/5 [==============================] - 0s 12ms/step - loss: 46.7458 - val_loss: 72.6256
Epoch 4/500
5/5 [==============================] - 0s 11ms/step - loss: 39.3288 - val_loss: 67.7948
Epoch 5/500
5/5 [==============================] - 0s 12ms/step - loss: 37.0374 - val_loss: 65.6379
Epoch 6/500
5/5 [==============================] - 0s 11ms/step - loss: 37.1516 - val_loss: 65.0785
Epoch 7/500
5/5 [==============================] - 0s 12ms/step - loss: 36.8489 - val_loss: 64.7685
Epoch 8/500
5/5 [==============================] - 0s 12ms/step - loss: 36.8235 - val_loss: 65.0353
Epoch 9/500
5/5 [==============================] - 0s 14ms/step - loss: 36.1383 - val_loss: 63.8180
Epoch 10/500
5/5 [==============================] - 0s 12ms/step - loss: 36.3750 - val_loss: 64.20

In [24]:
model = train_model(epochs=1_000, layer_sizes=[100, 100, 100, 100], X_train=X_train_scaled, y_train=y_train_log, X_valid=X_test_scaled, y_valid=y_valid_log, batch_norm=True, l1_l2=True)
evaluate_model(model, X_test_scaled, y_test, log_target=True)

Epoch 1/1000
5/5 [==============================] - 1s 55ms/step - loss: 135.8632 - val_loss: 121.4666
Epoch 2/1000
5/5 [==============================] - 0s 14ms/step - loss: 83.6910 - val_loss: 103.3548
Epoch 3/1000
5/5 [==============================] - 0s 13ms/step - loss: 62.7004 - val_loss: 95.3329
Epoch 4/1000
5/5 [==============================] - 0s 13ms/step - loss: 55.4414 - val_loss: 91.9281
Epoch 5/1000
5/5 [==============================] - 0s 13ms/step - loss: 52.5846 - val_loss: 90.4432
Epoch 6/1000
5/5 [==============================] - 0s 13ms/step - loss: 52.3933 - val_loss: 89.6609
Epoch 7/1000
5/5 [==============================] - 0s 12ms/step - loss: 51.3995 - val_loss: 89.5967
Epoch 8/1000
5/5 [==============================] - 0s 11ms/step - loss: 50.0060 - val_loss: 88.9638
Epoch 9/1000
5/5 [==============================] - 0s 12ms/step - loss: 49.7933 - val_loss: 87.6125
Epoch 10/1000
5/5 [==============================] - 0s 12ms/step - loss: 49.7284 - val_

In [ ]:
import time

# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Create a more complex DNN model
def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    # # this one was actually slower on GPU, batch size wasn't specified either
    # return tf.keras.models.Sequential([
    #     tf.keras.layers.Flatten(input_shape=(28, 28)),
    #     tf.keras.layers.Dense(128, activation='relu'),
    #     tf.keras.layers.Dropout(0.2),
    #     tf.keras.layers.Dense(10)
    # ])

# Train and evaluate the model, returning the time taken
def train_and_evaluate(model, x_train, y_train, x_test, y_test):
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    start_time = time.time()
    # model.fit(x_train, y_train, epochs=5)
    model.fit(x_train, y_train, epochs=5, batch_size=512)
    end_time = time.time()

    model.evaluate(x_test, y_test, verbose=2)
    
    return end_time - start_time

# Check if a GPU is available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
    with tf.device('/GPU:0'):  # train with GPU
        model = create_model()
        gpu_time = train_and_evaluate(model, x_train, y_train, x_test, y_test)
        print("Time taken to train with GPU: {:.2f} seconds".format(gpu_time))
        
    with tf.device('/CPU:0'):  # train with CPU
        model = create_model()
        cpu_time = train_and_evaluate(model, x_train, y_train, x_test, y_test)
        print("Time taken to train with CPU: {:.2f} seconds".format(cpu_time))
        
    speed_increase = (cpu_time - gpu_time) / cpu_time * 100
    print("GPU is {:.2f}% faster than CPU".format(speed_increase))
else:
    print("GPU not available, training with CPU only")
    model = create_model()
    cpu_time = train_and_evaluate(model, x_train, y_train, x_test, y_test)
    print("Time taken to train with CPU: {:.2f} seconds".format(cpu_time))